# Objective

This notebook takes the output from the previous step and looks up the latitudes and longitudes of each phone number based on the area/country codes.

It will load the country codes from the Internet and local files.

## Source Files Needed

These files are sourced from the public domain:

* us-area-code-geo.csv
* ca-area-code-geo.csv
* world_country_and_usa_states_latitude_and_longitude_values.csv

This file comes from the previous step:

* chat_area_codes_from_members.csv

## Output File

* chat_geo_data.csv


In [1]:
# Import some libraries
import numpy as np              # useful for many scientific computing in Python
import pandas as pd             # primary data structure library
from bs4 import BeautifulSoup   # For webscraping
import requests

## Load Chat Data

Load the data file that was generated from the previous step.


In [2]:
# Force pandas to load the 'Code' column as string type because we want to
# preserve the "+".  This is because we will use this later to determine if
# the code is US/Canada area code or a country code.
df = pd.read_csv("chat_area_codes_from_members.csv", dtype={'Code': np.string_})
df.head()

,Code
0,201
1,203
2,206
3,219
4,236


In [3]:
# Check the data types
df.dtypes

Code    object
dtype: object

## Load Geo Data

Load the geo data that contains the latitudes and longitudes.

In [4]:
# Load the country calling codes from the Internet
countr_code_url = "https://countrycode.org/"
response = requests.get(countr_code_url)
soup = BeautifulSoup(response.content, "html.parser")
soup.title

<title>Country Codes, Phone Codes, Dialing Codes, Telephone Codes, ISO Country Codes</title>

In [5]:
# Search through the table and extract the country by calling codes.

rows = soup.find("table").find("tbody").find_all("tr")  # Extract all rows from the table on the page
country_codes = {}                                      # Initialize an empty dictionary to store the country + code
for row in rows:                                        # Loop through all the rows
    cells = row.find_all("td")                          # Extract the cells from the row
    country = cells[0].get_text()                       # First cell is the country
    code = cells[1].get_text()                          # Second cell is the calling code
    country_codes[code] = country                       # Save code/country to the dictionary

# Note: the 'calling code' is the key and the 'country' is the value.  This is
# because we want to look up the country by code later on.

In [6]:
# Display the first 5 elements in the dictionary to verify we did it right
first_few = dict(list(country_codes.items())[:5])
first_few

{'93': 'Afghanistan',
 '355': 'Albania',
 '213': 'Algeria',
 '1-684': 'American Samoa',
 '376': 'Andorra'}

In [7]:
# Load US geo data
df_area_codes = pd.read_csv("us-area-code-geo.csv")
df_area_codes.head()

,area_code,latitude,longitude
0,201,40.838850,-74.045678
1,202,38.895110,-77.036370
2,203,41.291798,-73.122453
3,205,33.427671,-86.886473
4,206,47.564027,-122.348976


In [8]:
# Load Canada geo data
df_ca_area_codes = pd.read_csv("ca-area-code-geo.csv")
df_ca_area_codes.head()

,area_code,latitude,longitude
0,204,51.203034,-98.729935
1,226,43.233831,-81.230922
2,236,50.432725,-121.515369
3,249,46.041823,-80.003286
4,250,50.837259,-121.841520


In [9]:
# Load the world geo data
df_codes = pd.read_csv("world_country_and_usa_states_latitude_and_longitude_values.csv")
df_codes.head()

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


## Determine Latitudes/Longitudes

Now that we loaded the various files, we can now go through the input file and figure out the latitudes/longitudes for each member.

#### Step 1: Determine Country

Take the area/country calling codes and figure out what country it belongs to.

In [10]:
# Function to return the country based on the calling code
def getCountry(code):
  if code[0] == "+":                                      # Does the code begin with "+"?
    new_code = code[1:]                                   # Yes, then extract the code without the "+"
    if new_code in country_codes:                         # Is this a valid country code?
      return country_codes[new_code]                      # Yes, return the country
  int_code = int(code)                                    # Convert string to integer for the next section
  if int_code in df_ca_area_codes['area_code'].values:    # No "+" or invalid code, is the code in Canada?
    return 'Canada'                                       # Yes, return 'Canada'
  elif int_code in df_area_codes['area_code'].values:     # Not Canada, is it in the US?
    return 'United States'                                # Yes, return 'United States'

  # Country/Area code wasn't found!  Let's start trimming it from the back
  # and see if we can find it.
  if len(code) > 1:                                       # Is the length greater than 1?
    return getCountry(code[:-1])                          # Yes, then call this function again with a shorter string (drop the last character)
  return None                                             # Still can't find it!  Return None instead!

# If this function can't find the code, either the code really doesn't exist (bad input data),
# or we need to update our source tables for country/area codes.

In [11]:
# Test the function with some values
test_code = '+49'
print('{} belongs to {}'.format(test_code, getCountry(test_code)))
test_code = '201'
print('{} belongs to {}'.format(test_code, getCountry(test_code)))
test_code = '+316'
print('{} belongs to {}'.format(test_code, getCountry(test_code)))

# This is an edge case.  The input file contains this value.
test_code = '+447956'
print('{} belongs to {}'.format(test_code, getCountry(test_code)))

+49 belongs to Germany
201 belongs to United States
+316 belongs to United States
+447956 belongs to United Kingdom


In [12]:
df2 = df.copy()                                                         # Make a copy of the original data
df2['Country'] = df2.apply(lambda row: getCountry(row['Code']), axis=1) # Create a new 'Country' column based on the country/area code

In [13]:
# Check to make sure there are no nulls
df2.isnull().sum()

Code       0
Country    0
dtype: int64

In [14]:
# Check the value counts just to be curious
df2['Country'].value_counts()

United States           68
Canada                  10
Mexico                  10
India                    5
United Arab Emirates     4
Colombia                 4
Saudi Arabia             3
Italy                    3
Ghana                    2
Switzerland              2
United Kingdom           2
Ecuador                  2
Germany                  2
Chile                    2
Turkey                   1
Argentina                1
Bahrain                  1
Costa Rica               1
Peru                     1
Panama                   1
Poland                   1
Spain                    1
France                   1
Belgium                  1
Netherlands              1
Mali                     1
Qatar                    1
Name: Country, dtype: int64

#### Step 2: Determine Latitudes/Longitudes

In [15]:
# Function to return latitude/longitude based on the country and/or area code of the row
def getLatitudeLongitude(row):
  ret_val = None                                              # Start with a blank/empty value
  match row['Country']:                                       # Check 'Country'
    case 'United States':                                     # Is it 'United States'?
      area_code = int(row['Code'])                            # Yes, convert the 'Code' from string to integer and get the coordinates for it
      retVal = df_area_codes[['latitude', 'longitude']].loc[df_area_codes['area_code'] == area_code]

    case 'Canada':                                            # Is it 'Canada'?
      area_code = int(row['Code'])                            # Yes, convert the 'Code' from string to integer and get the coordinates for it
      retVal = df_ca_area_codes[['latitude', 'longitude']].loc[df_ca_area_codes['area_code'] == area_code]

    case _:                                                   # Must be a country calling code; get the coordinates for the country
      retVal = df_codes[['latitude', 'longitude']].loc[df_codes['country'] == row['Country']]

  if len(retVal) == 1:                                        # Found exactly one row?  Should always be true.
    return pd.Series([retVal.iloc[0, 0], retVal.iloc[0, 1]])  # Yes, return latitude/longitude
  return None                                                 # No, return None

In [16]:
# Test the function with the first row of the dataset
row = df2.iloc[0]
ret = getLatitudeLongitude(row)
print('Code {} is located at {}, {}'.format(row['Code'], ret[0], ret[1]))

# Test the function with the last row of the dataset
row = df2.iloc[-1]
ret = getLatitudeLongitude(row)
print('Code {} is located at {}, {}'.format(row['Code'], ret[0], ret[1]))

Code 201 is located at 40.83885, -74.04567813
Code +974 is located at 25.354826, 51.183884


In [17]:
# Create a copy of the dataset
df3 = df2.copy()

# Now, for each row, call our function and save the data to new columns
df3[['latitude', 'longitude']] = df3.apply(getLatitudeLongitude, axis=1)
df3.head()

,Code,Country,latitude,longitude
0,201,United States,40.838850,-74.045678
1,203,United States,41.291798,-73.122453
2,206,United States,47.564027,-122.348976
3,219,United States,41.566263,-87.263600
4,236,Canada,50.432725,-121.515369


In [18]:
# Verify that there are no null rows
df3[df3.isnull().any(axis=1)]

,Code,Country,latitude,longitude


In [19]:
# Save the data to a CSV file
df3.to_csv("chat_geo_data.csv", index=False)